In [198]:

import pandas as pd
import numpy as np
import hjson

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import re

# nltk.download('stopwords')
# nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from get_dataframes import GetDataframes

from scipy.sparse import hstack

pd.set_option('display.max_columns', None)

In [199]:
with open("config.hjson") as fp:
    config = hjson.load(fp)
datas = GetDataframes(config)

df = datas.get_machine_learning_dataframe(
    "machine_learning",
    # modify=True
)

2023-11-10 09:25:53 INFO     Parquet loaded ! Importing machine_learning...
2023-11-10 09:25:53 INFO     Dataframe machine_learning ready to use!


In [200]:
def full_lower(
    text: str
):
    return text.replace(" ", "").replace("-", "").lower()

tt = (
    ("titre_genres", "titre_genres"),
    ("actors", "actors"),
    ("titre_genres", "titre_genres"),
    ("directors", "directors"),
)
for t in tt:
    df[t[0]] = df[t[1]].apply(full_lower)


# df["overview"] = df["overview"].apply(lambda x : x.replace(" ", ","))


In [201]:
df

,titre_id,titre_str,titre_genres,rating_avg,rating_votes,actors,directors,overview
0,tt0035423,Kate & Leopold,"comedy,fantasy,romance",6.40,87897,"megryan,hughjackman,lievschreiber,breckinmeyer",jamesmangold,scientist ex boyfriend discovers portal travel...
1,tt0052832,The Fugitive Kind,"drama,romance",7.10,6914,"annamagnani,marlonbrando,joannewoodward,mauree...",sidneylumet,val xavier drifter obscure origin arrives smal...
2,tt0052997,The League of Gentlemen,"comedy,crime,drama",7.20,5105,"jackhawkins,rogerlivesey,nigelpatrick,richarda...",basildearden,involuntarily retired colonel hyde recruit sev...
3,tt0053459,Eyes Without a Face,"drama,horror",7.60,34278,"pierrebrasseur,alexandrerignault,alidavalli,ju...",georgesfranju,dr g nessier riddled guilt accident caused dis...
4,tt0053579,Late Autumn,"comedy,drama",7.90,5854,"setsukohara,keijisada,marikookada,yôkotsukasa",yasujirôozu,woman daughter forced contend increasing press...
...,...,...,...,...,...,...,...,...
7651,tt9873892,They Cloned Tyrone,"comedy,mystery,scifi",6.60,33197,"kiefersutherland,jamiefoxx,johnboyega,teyonahp...",jueltaylor,series eerie event thrust unlikely trio onto t...
7652,tt9883996,Dream Horse,"biography,comedy,drama",6.90,6589,"alandavid,lyndabaron,owenteale,tonicollette",euroslyn,inspiring true story dream alliance unlikely r...
7653,tt9893250,I Care a Lot,"comedy,crime,drama",6.40,141372,"diannewiest,rosamundpike,peterdinklage,eizagon...",jblakeson,court appointed legal guardian defrauds older ...
7654,tt9900782,Kaithi,"action,adventure,crime",8.50,39546,"narain,karthi,georgemaryan,arjundas",lokeshkanagaraj,dilli convicted criminal parole meet daughter ...


In [202]:
print(df[df["titre_str"].str.contains("The Avengers")].to_markdown())

|      | titre_id   | titre_str    | titre_genres   |   rating_avg |   rating_votes | actors                                                    | directors   | overview                                                                                                                                                                                                                |
|-----:|:-----------|:-------------|:---------------|-------------:|---------------:|:----------------------------------------------------------|:------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 4330 | tt0848228  | The Avengers | action,scifi   |            8 |        1437952 | robertdowneyjr.,scarlettjohansson,chrisevans,jeremyrenner | josswhedon  | unexpected enemy emerges threatens global safety security nick fury director interna

In [203]:
print(df[df["titre_id"].str.contains("tt2395417")].to_markdown())


|      | titre_id   | titre_str   | titre_genres   |   rating_avg |   rating_votes | actors                                             | directors      | overview                                           |
|-----:|:-----------|:------------|:---------------|-------------:|---------------:|:---------------------------------------------------|:---------------|:---------------------------------------------------|
| 6148 | tt2395417  | Still Life  | drama          |          7.4 |           8708 | karendrury,eddiemarsan,joannefroggatt,andrewbuchan | ubertopasolini | council case worker look relative found dead alone |


In [204]:
# feat = [
#     'titre_str',
#     'titre_genres',
#     # 'rating_avg',
#     # 'rating_votes',
#     "actors",
#     'directors',
# ]
# # df.columns.to_list()
# feat

# for f in feat:
#     df[f] = df[f].replace(" ", "")

In [205]:
def combine(r):
    return r["titre_genres"]+" "+r["directors"]+" "+r["overview"]+" "+r["actors"] #r["titre_str"]+" "+

df["one_for_all"] = df.apply(combine, axis=1)

In [206]:
def choice(movie: str):
    if movie == "genres":
        poids_ = {
            # "titre_str":    0.2,
            "titre_genres": 0.2,
            "actors":       0.2,
            "directors":    0.2,
            "overview":     0.2,
        }


In [207]:
def titre_index(titre: str):
    return df[df.titre_str == titre].index[0]

def director_index(director: str):
    return df[df.directors.str.contains(director)].index[0]

def actor_index(actor: str):
    return df[df.actors.str.contains(actor)].index[0]

def idx_titre(idx: int):
    return df[df.index == idx]["titre_str"].values[0]

def idx_actor(idx: int):
    return df[df.index == idx]["actors"].values[0]

def idx_titre_id(idx: int):
    return df[df.index == idx]["titre_id"].values[0]

# def idx_title(idx):
#     return df[df.index == idx]["titre_str"].values[0]

In [214]:
# tidf = TfidfVectorizer()
# combined_matrix = tidf.fit_transform(
#     df["titre_str"]+" "+df["actors"]+" "+df["titre_genres"]+" "+df["directors"]+" "+df["overview"]
# )
# tidf = TfidfVectorizer()
# combined_matrix = tidf.fit_transform(
#     df["one_for_all"]+" "+df["titre_str"]
# )
top = 10

poids_ = {
    "titre_str":    0.2,
    "titre_genres": 0.2,
    "actors":       0.2,
    "directors":    0.2,
    "overview":     0.2,
    # "titre_genres": 2,
    # "actors":       1.5,
    # "directors":    1.5,
    # "overview":     2.5,
}


full_matrix = []
for col, poids in poids_.items():
    tfidf_ = TfidfVectorizer()
    matrix_ = tfidf_.fit_transform(df[col]) * poids
    full_matrix.append(matrix_)
combined_matrix = hstack(full_matrix)
cosine = cosine_similarity(combined_matrix)
movies = "The Avengers"
mov_idx = titre_index(movies)
similar = cosine[mov_idx]
similar1 = list(enumerate(cosine[mov_idx]))
sim_scores = sorted(similar1, key=lambda x: x[1], reverse=True)
sim_mov_idx = similar.argsort()[::-1][1:top+1]
same_movies = df.loc[sim_mov_idx, "titre_str"]
ttconst = df.loc[sim_mov_idx, "titre_id"]
sim_scores[1:top+1]
score = [i[1] for i in sim_scores]

print(f"Top 10 similar movies to {movies} are :\n")
for movies, tt,  score in zip(same_movies, ttconst, score):
    print(f"Movie : {movies} | id : {tt} | score : {np.round(score, 4)}")


Top 10 similar movies to The Avengers are :

Movie : Still Life | id : tt2395417 | score : 1.0
Movie : The Legend of Maula Jatt | id : tt4139928 | score : 0.6051
Movie : Serenity | id : tt0379786 | score : 0.3859
Movie : The Walk | id : tt3488710 | score : 0.3753
Movie : Captain Marvel | id : tt4154664 | score : 0.3408
Movie : Like Someone in Love | id : tt1843287 | score : 0.2974
Movie : Chronicle | id : tt1706593 | score : 0.2832
Movie : Rent-A-Pal | id : tt12274228 | score : 0.2611
Movie : Captain America: The First Avenger | id : tt0458339 | score : 0.2491
Movie : The Island | id : tt0399201 | score : 0.2317


In [209]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer

movies = "The Avengers"

cv = CountVectorizer()
count_matrix = cv.fit_transform(df['one_for_all'])
movie_user_likes = movies
movie_index = titre_index(movie_user_likes)

knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(count_matrix)
dist, indices = knn_model.kneighbors(count_matrix[movie_index], n_neighbors=15)

print(f"Top 10 similar movies to {movie_user_likes} are :\n")
for index, dis in zip(indices.flatten()[1:], dist.flatten()[1:]):
    print(f"Movie : {idx_titre(index)} | id : {idx_titre_id(index)} | score : {np.round(dis, 4)}")


Top 10 similar movies to The Avengers are :

Movie : Still Life | id : tt2395417 | score : 0.7633
Movie : Enemy Mine | id : tt0089092 | score : 0.8129
Movie : The Blood of Heroes | id : tt0094764 | score : 0.831
Movie : Like Someone in Love | id : tt1843287 | score : 0.8318
Movie : Gandhi Godse Ek Yudh | id : tt13801462 | score : 0.8343
Movie : The Walk | id : tt3488710 | score : 0.8355
Movie : The Last Starfighter | id : tt0087597 | score : 0.8376
Movie : Goodbye Berlin | id : tt4911940 | score : 0.8416
Movie : Chronicle | id : tt1706593 | score : 0.8431
Movie : Everything Everywhere All at Once | id : tt6710474 | score : 0.8457
Movie : Serenity | id : tt0379786 | score : 0.8471
Movie : Star Trek VI: The Undiscovered Country | id : tt0102975 | score : 0.8482
Movie : Better Days | id : tt9586294 | score : 0.8485
Movie : Pirates of the Caribbean: On Stranger Tides | id : tt1298650 | score : 0.8525


In [210]:

# random_model = RandomForestClassifier().fit(combined_matrix, df["titre_genres"])
# for idx, movies in zip(sim_mov_idx, same_movies):
#     mo = combined_matrix[idx].toarray()
#     prediction = random_model.predict(mo)[0]
#     confidence = np.max(random_model.predict_proba(mo))

#     print("movie :", movies)
#     print("confidence :", confidence)

In [211]:
# Movie : Avengers: Age of Ultron | id : tt2395427 | score : 1.0
# Movie : The Gray Man | id : tt1649418 | score : 0.2522
# Movie : Avengers: Endgame | id : tt4154796 | score : 0.1226
# Movie : Snowpiercer | id : tt1706620 | score : 0.1209
# Movie : Kingsman: The Secret Service | id : tt2802144 | score : 0.1117
# print(f"Top 10 similar movies to {movie_user_likes} are :\n")
#

In [212]:
# poids_ = {
#     "titre_str": 0.1,
#     "titre_genres": 0.1,
#     "directors": 0.2,
#     "overview": 0.2,
#     "actor_1": 0.15,
#     "actor_2": 0.1,
    # "actor_3": 0.025,
#     "actor_4": 0.025,
# }

# full_matrix = []
# for col, poids in poids_.items():
#     tfidf_ = TfidfVectorizer()
#     matrix_ = tfidf_.fit_transform(df[col]) * poids
#     full_matrix.append(matrix_)

# combined_matrix = hstack(full_matrix)

# cosine = cosine_similarity(combined_matrix)

# movies = movie
# mov_idx = title_index(movies)
# similar = cosine[mov_idx]
# similar1 = list(enumerate(cosine[mov_idx]))
# sim_scores = sorted(similar1, key=lambda x: x[1], reverse=True)

# sim_mov_idx = similar.argsort()[::-1][1:11]
# same_movies = df.loc[sim_mov_idx, "titre_str"]
# sim_scores[1:11]
# score = [i[1] for i in sim_scores]
# for movies, score in zip(same_movies, score):
#     print(movies, score)


# # random_model = RandomForestClassifier().fit(matrix, df["titre_genres"])
# # for idx, movies in zip(sim_mov_idx, same_movies):
# #     mo = matrix[idx].toarray()
# #     prediction = random_model.predict(mo)[0]
# #     confidence = np.max(random_model.predict_proba(mo))

# #     print("movie :", movies)
# #     print("confidence :", confidence)